# Unsupervised Network Anomoly Detection System

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import tensorflow as tf
from tensorflow import keras
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Network Anomaly Dectection/Test.csv')
test

,0,tcp,private,REJ,0.1,0.2,0.3,0.4,0.5,0.6,...,0.04.1,0.06.1,0.00.3,0.00.4,0.00.5,0.00.6,1.00.2,1.00.3,neptune,21
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune,21
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal,21
2,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint,15
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan,11
4,0,tcp,http,SF,267,14515,0,0,0,0,...,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal,21
22539,0,tcp,http,SF,317,938,0,0,0,0,...,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal,21
22540,0,tcp,http,SF,54540,8314,0,0,0,2,...,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back,15
22541,0,udp,domain_u,SF,42,42,0,0,0,0,...,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal,21


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Network Anomaly Dectection/Train.csv')
train

,0,tcp,ftp_data,SF,491,0.1,0.2,0.3,0.4,0.5,...,0.17,0.03,0.17.1,0.00.6,0.00.7,0.00.8,0.05,0.00.9,normal,20
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125968,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125970,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [ ]:
X = pd.DataFrame(train)  # You can just pass the array directly; columns are optional
y = pd.DataFrame(test)   # Similar to above
y_train = train['normal']
y_test = test['neptune']

X = X.drop('normal', axis=1)
y = y.drop('neptune', axis=1)

X.columns = [f'Column_{i}' for i in range(1, X.shape[1] + 1)]
y.columns = [f'Column_{i}' for i in range(1, y.shape[1] + 1)]

# Combine the DataFrames (X_train and X_test must have the same columns for this to make sense)
combined = pd.concat([X,y],  ignore_index=True)
# Combine target variables vertically
Y = pd.concat([y_train, y_test], ignore_index=True)
combined, Y


(        Column_1 Column_2  Column_3 Column_4  Column_5  Column_6  Column_7  \
 0              0      udp     other       SF       146         0         0   
 1              0      tcp   private       S0         0         0         0   
 2              0      tcp      http       SF       232      8153         0   
 3              0      tcp      http       SF       199       420         0   
 4              0      tcp   private      REJ         0         0         0   
 ...          ...      ...       ...      ...       ...       ...       ...   
 148510         0      tcp      smtp       SF       794       333         0   
 148511         0      tcp      http       SF       317       938         0   
 148512         0      tcp      http       SF     54540      8314         0   
 148513         0      udp  domain_u       SF        42        42         0   
 148514         0      tcp    sunrpc      REJ         0         0         0   
 
         Column_8  Column_9  Column_10  ...  Colum

## Data Processing
* We will one hot encode all categorical data assuming unqiue values are not large
* Perform PCA to reduce dimensions and check for releveant correlations

In [ ]:
def processing_with_pca(df, threshold=0.8, explained_variance_ratio=0.95, test=0):

    categorical_cols = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_cols)

    # Standardize the numerical features
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df)

    pca = PCA()
    pca_result = pca.fit_transform(scaled_data)

    explained_variance = pca.explained_variance_ratio_.cumsum()
    num_components = next(i for i, total_var in enumerate(explained_variance) if total_var >= explained_variance_ratio) + 1


    pca = PCA(n_components=num_components)
    reduced_data = pca.fit_transform(scaled_data)

    # Convert PCA result to DataFrame
    pca_df = pd.DataFrame(data=reduced_data, columns=[f'PC{i+1}' for i in range(num_components)])

    return pca_df

# Example Usage
# Assuming 'train' is your DataFrame
data = processing_with_pca(combined)
data

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC81,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89,PC90
0,0.071283,1.470098,-3.697859,-0.913287,1.379917,1.395960,-4.113514,-0.131239,-2.383079,1.363110,...,0.101313,-0.042418,0.689471,0.120706,0.058776,0.038251,-0.007803,0.511946,-1.030928,0.287915
1,-4.411159,-2.187540,-0.077708,0.147408,-0.129768,-0.091659,-0.353660,-0.060102,0.004338,-0.779486,...,-0.032925,-0.042602,-1.150620,0.117019,-0.223216,0.071024,-0.212094,0.244848,-0.020508,-0.299004
2,2.674705,-0.936700,1.873738,-0.016094,-0.404822,-0.486040,0.095203,0.307242,-0.249458,-0.351224,...,0.012882,-0.021822,0.055492,-0.014379,0.128999,0.051099,-0.031276,0.006172,0.027635,0.257086
3,2.532893,-0.756013,1.665953,-0.835485,-0.047372,-0.400564,0.353824,0.521146,-0.241736,-0.083207,...,0.025980,-0.028779,0.099782,0.032714,-0.032140,0.000800,0.034772,0.089128,0.000596,-0.396493
4,-2.568172,5.620857,0.957598,-0.667339,-0.330956,-0.623004,1.342440,-0.829821,0.004763,-0.896728,...,-0.052780,0.080805,-0.980015,-0.005754,-0.180615,-0.030640,-0.256028,-0.014657,-0.552344,0.545043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148510,2.166218,-0.578713,1.458068,0.136987,-0.096528,0.075563,-0.694728,-0.284827,-0.779724,1.088859,...,0.047629,-0.102700,0.041192,0.060607,-0.119559,-0.150950,-0.007277,-0.196514,-0.155571,0.610632
148511,2.705555,-0.754141,1.775433,-0.463213,-0.207152,-0.421555,0.181191,0.292949,-0.227506,-0.153719,...,0.022771,-0.032320,0.100713,0.026906,-0.010280,0.021497,0.022326,0.086492,0.008511,-0.285720
148512,2.570602,-0.458221,1.437327,-0.210097,0.157604,0.423448,0.226179,0.386896,0.305470,0.007494,...,0.053586,-0.047002,0.398951,-0.001540,-0.216511,-0.056315,0.139869,-0.380838,0.824185,-0.990151
148513,2.539528,-0.564222,-2.151772,-2.959587,1.081191,-0.140183,-0.674149,-2.590995,1.305996,-0.895713,...,-0.016041,0.039781,-0.186919,-0.025364,-0.109848,-0.023953,0.033697,0.029976,-0.121443,0.017736


In [ ]:
y = pd.Series([1 if x != 'normal' else 0 for x in Y])
X_train, X_test, y_train, y_test = train_test_split(data,y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test

(             PC1       PC2       PC3       PC4       PC5       PC6       PC7  \
 133863  2.842495 -0.749825  1.897816 -0.302384 -0.265588 -0.427652  0.131265   
 8340    3.118599 -0.633902  1.482470  0.276363 -0.483304 -0.450351 -0.146691   
 48286   1.518381 -0.451656  0.737194  0.227649  0.099774  0.413177 -1.067975   
 54230   0.714653  0.175995 -3.192496 -1.956964  1.591363  1.179106 -2.667475   
 145508  3.057984 -0.724365  1.752504  0.046013 -0.446614 -0.498621  0.106321   
 ...          ...       ...       ...       ...       ...       ...       ...   
 119879 -4.390550 -2.212284 -0.018368  0.013232 -0.154175 -0.199947 -0.296633   
 103694 -4.591861 -2.356115  0.153125  0.322386 -0.254618 -0.138029  0.222259   
 131932 -0.614338  1.623349 -3.489249 -0.858854  1.095838  1.119692 -3.482607   
 146867 -2.731308  5.482710  0.907508 -0.392881 -0.389159 -0.530968  2.012058   
 121958  2.507150 -0.768041  1.829392 -0.798039 -0.026180 -0.349778  0.192502   
 
              PC8       PC

In [ ]:
'''
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier()
}

results = {}

# Loop through the models and compute cross-validation scores for each
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    results[name] = cv_scores

# Display the results
for name, scores in results.items():
    print(f"{name}: Mean Accuracy = {np.mean(scores):.2f} ± {np.std(scores):.2f}")
    '''

'\nmodels = {\n    \'Logistic Regression\': LogisticRegression(max_iter=1000),\n    \'Random Forest\': RandomForestClassifier()\n}\n\nresults = {}\n\n# Loop through the models and compute cross-validation scores for each\nfor name, model in models.items():\n    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring=\'accuracy\')\n    results[name] = cv_scores\n\n# Display the results\nfor name, scores in results.items():\n    print(f"{name}: Mean Accuracy = {np.mean(scores):.2f} ± {np.std(scores):.2f}")\n    '

In [ ]:
X_train = tf.constant(X_train, dtype=tf.float32)
y_train = tf.constant(y_train, dtype=tf.float32)
X_test = tf.constant(X_test, dtype=tf.float32)
y_test = tf.constant(y_test, dtype=tf.float32)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Use sigmoid for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=32)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9773 - loss: 0.0687
Epoch 2/5
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - accuracy: 0.9928 - loss: 0.0207
Epoch 3/5
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9932 - loss: 0.0189
Epoch 4/5
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.9944 - loss: 0.0159
Epoch 5/5
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.9951 - loss: 0.0127
929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9944 - loss: 0.0143
Test Loss: 0.014490894973278046, Test Accuracy: 0.9937043190002441


In [ ]:
X_test
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # Convert probabilities to binary classes

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Print classification report
print(classification_report(y_test, y_pred_classes))


929/929 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step
Confusion Matrix:
[[15201   156]
 [   31 14315]]
              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99     15357
         1.0       0.99      1.00      0.99     14346

    accuracy                           0.99     29703
   macro avg       0.99      0.99      0.99     29703
weighted avg       0.99      0.99      0.99     29703

